# Simple Run Test

My agents kept producing errors, so this is a very basic setup to test them.

The current agent is defined below:

In [ ]:
%%writefile test_agent.py

import random

ROCK, PAPER, SCISSORS = 0, 1, 2
OPTIONS = (ROCK, PAPER, SCISSORS)

def make_choice(step, score, oppo, mine):
    if step == 0:
        return random.choice(OPTIONS)
    if step == 1:
        # expect different sign than last time
        return oppo[-1] + 2
    # default to random choice
    return random.choice(OPTIONS)

op_choices = []
my_choices = []
my_score = 0

def agent_call(observation, configuration):
    global op_choices
    global my_choices
    global my_score
    # log last rounds opponent choice before making decision
    if observation.step > 0 and observation.lastOpponentAction in OPTIONS:
        op_choices.append(observation.lastOpponentAction)
        if my_choices[-1] == (op_choices[-1]+1)%3:
            my_score += 1
        elif my_choices[-1] == (op_choices[-1]-1)%3:
            my_score -= 1
        elif my_choices[-1] != op_choices[-1]:
            print(my_choices[-1], op_choices[-1])
            raise Exception("Tie but not the same choice!")
    choice = make_choice(observation.step, my_score, op_choices, my_choices)
    choice %= 3 # allows to add 1 to an option to get the beating option
    my_choices.append(choice)
    return choice

A simple benchmark agent:

In [ ]:
%%writefile copy_opponent.py
    
def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return 0

In [ ]:
from kaggle_environments import make
env = make("rps", configuration={"episodeSteps": 1000})

In [ ]:
# make a test run and show logs for debugging
env.reset()
env.run(["test_agent.py", "copy_opponent.py"])
for entry in env.logs:
    for subentry in entry:
        for key,val in subentry.items():
            print(key, ':')
            print(val)
        print(80*'-')

In [ ]:
# see rendered outcome of test run
env.render(mode="ipython", width=550, height=500)